# iEBPR simulation example

iEBPR is an agent-based model for simulating PAO-GAO competition.

# Installation

Installation (building from source) requires:

* setuptools >= 61.0.0
* numpy >= 1.15.0
* C++ compiler (supporting C++11)

Then build & install with

```bash
pip install ./
```

# Example run

## 1. Setup simulation basics

In [ ]:
import numpy
import matplotlib.pyplot

import iebpr
from iebpr import Simulation, EnvState, SbrPhase, SbrStage, RandType, \
	AgentSubtype, RandConfig, StateRandConfig, TraitRandConfig

simulation = Simulation(seed=0, pcontinuous=True, timestep=1e-5)

`seed` sets the random seed, default is 0.

`pcontinuous` sets the simulation type, `True` for pseudo-continuous and `False`
	for discrete-time.
	
Both discrete-time and pseudo-continuous simulations process a fixed number of agents during each timestep (total number of agents).  Discrete-time simulation traverses through all existing agents in each timestep and updates the environmental states all together thereafter. On the other hand, pseudo-continuous simulation randomly selects agents to process during each time step and immediately update envrionmental states after each agent's action. In general, pseudo-continuous is closer to a continuous state simulation and therefore more precise, but can take significantly longer to finish.

`timestep` sets the simulation timestep, i.e. the scale of `dt` to calculate agent and environment state changes.

The above configuration is equivalent to:

```python
simulation = Simulation()
simulation.seed = 0
simulation.pcontiuous = True
simulation.timestep = 1e-5
```

Likewise, both ways are compatible with most of other data objects in `iebpr`.

## 2. Setup SBR workflow

### 2.1 Setup initial environment states

These are the initial state of the simulated reactor at the beginning of the simulation. They can be configured with:

In [ ]:
simulation.init_env = EnvState(
	volume=40, # in L
	vfa_conc=0, # in mgCOD/L
	op_conc=0, # in mgP/L
)

### 2.2 Setup SBR control

SBR is controlled in stages, and stages are composed by phases. Each phase sets the SBR states like inflow, outflow, or aeration to a duration (`time_len`). The time unit used by `iebpr` is day, so unit conversions may be necessary. The explicit way to create a phase is:

In [ ]:
inflow_phase = SbrPhase(
	time_len=30 / 60 / 24, # 30min->day, the duration
	inflow_rate=5 / (30 / 60 / 24), # input 5L/30min -> L/day
	inflow_vfa_conc=200, # in mgCOD/L
	inflow_op_conc=25, # in mgP/L
	aeration=False,
)

Then create more phases:

In [ ]:
anaerobic_phase = SbrPhase(
	time_len=90 / 60 / 24,
	aeration=False,
)

aerobic_phase = SbrPhase(
	time_len=180 / 60 / 24,
	aeration=True,
)

withdraw_phase = SbrPhase(
	time_len=30 / 60 / 24,
	# withdraw discards biomass based on outflow volume
	# vfa, op, and biomass conc. remain the same during withdraw
	withdraw_rate=1 / (30 / 60 / 24),
	aeration=True,
)

outflow_phase = SbrPhase(
	time_len=30 / 60 / 24,
	# outflow reduces reactor living volume without reducing biomass
	# this will result in biomass conc. increase
	outflow_rate=4 / (30 / 60 / 24),
	aeration=True,
)

Finally, compose all phases as a stage, and add the stage to the simulation:

In [ ]:
# create stage from phases configs
stage = SbrStage(
	n_cycle=100,
	cycle_phases=[
		inflow_phase,
		anaerobic_phase,
		aerobic_phase,
		withdraw_phase,
		outflow_phase,
	]
)

# check if flow is balanced
print("stage flow balanced:", stage.is_flow_balanced())

# add to SBR control
simulation.append_sbr_stage(stage)

When a stage is on, the SBR will be controlled to cycle through all of the stage's phases in the added order. In this example, it will go through inflow, anaerobic, aerobic, withdraw, and outflow phases in a row to complete a said "cycle". Each stage will complete `n_cycle` number of cycles before moving to the next stage. The simulation will stop upon finishing the last cycle of the last stage.

It's usually assumed that the input volume is equal to the output volume, leaving the reactor's volume unchanged upon finishing of each cycle. In this case, the configured stage has 5L input volume from `inflow_phase`, and 5L output volume (as a combination of 1L in `withdraw_phase` and 4L in `outflow_phase`). To explicitly check this, do:

```python
stage.is_flow_balanced() # return True if balanced
```

In addition,

```python
simulation.is_flow_balanced()
```

can be invoked to check if *all* stages have balanced flow.

## 3. Setup biomass

In `ibepr`, the biomass agents are randomly generated to emulate the heterogeneity. Therefore, the agent states and traits are configured through randomizing parameters rather than explicitly setting the values. The randomizing parameters can be stored in `json` files similar to examples illustrated in `doc/example.gao_trait.json`. Alternatively, `iebpr.agent_template.get_template()` can be invoked as a good starting point, for example:

```
oho_trait_template = iebpr.agent_template.get_template("oho_trait")
```

The templates give a hint at what fields will be used in calculations. However, they do *NOT* imply that the template values are meaningful or defaults. Calibration are *ALWAYS* needed to simulate real experimental data.

`iebpr` provides three types of agent subtypes:

* `AgentSubtype.pao`
* `AgentSubtype.gao`
* `AgentSubtype.oho`

each corresponds to a specific suite of bioprocess equations. To add a new agent subtype to the simulation, call:

In [ ]:
# pao
simulation.add_agent_subtype(
	AgentSubtype.pao,
	n_agent=100,
	state_cfg=StateRandConfig(
		biomass=RandConfig(RandType.normal, mean=100, stddev=10),
		glycogen=RandConfig(RandType.normal, mean=10, stddev=2),
		pha=RandConfig(RandType.normal, mean=20, stddev=2),
		polyp=RandConfig(RandType.normal, mean=15, stddev=2),
	),
	trait_cfg=iebpr.agent_template.randconfig_from_template_json(
		"example.pao_trait.json"
	),
)

The built-in function `iebpr.agent_template.randconfig_from_template_json()` reads the above-said json file and return a corresponding state or trait randomizer configuration set.

Importantly, the `state` values are initial values at bulk-level. In the above example, the added pao subtype will have 100mgCOD biomass/L at the beginning of the simulation. Since it is configured to have 100 agents, each agent will be 1mgCOD biomass/L on average.

Similarly, add other subtypes to the simulation:

In [ ]:
# gao
simulation.add_agent_subtype(
	AgentSubtype.gao,
	n_agent=100,
	state_cfg=StateRandConfig(
		biomass=RandConfig(RandType.normal, mean=100, stddev=10),
		glycogen=RandConfig(RandType.normal, mean=10, stddev=2),
		pha=RandConfig(RandType.normal, mean=20, stddev=2),
	),
	trait_cfg=iebpr.agent_template.randconfig_from_template_json(
		"example.gao_trait.json"
	),
)

# oho
simulation.add_agent_subtype(
	AgentSubtype.oho,
	n_agent=50,
	state_cfg=StateRandConfig(
		biomass=RandConfig(RandType.normal, mean=100, stddev=10),
	),
	trait_cfg=iebpr.agent_template.randconfig_from_template_json(
		"example.oho_trait.json"
	),
)

## 4. Setup recording

Recording controls how and when to record the environment and agent states during the simulation run. There are two types of recordings:

* state recording: records environmental states and bulk-level agent states. This recording can be compared with bulk-level measurements such as vfa, op, and biomass concentrations.
* snapshot recording: records all states of all individual agents. This recording shows the distribution of glycogen, pha, and polyp over each subtype and can be compared with single-cell level measurements.

To setup the recording timepoints, run:

In [ ]:
last_cycle_start_time = simulation.total_time_len - stage.cycle_time_len

# state_rec_timepoints
# record the bulk-level environment and agent states during the last cycle
timepoints = numpy.linspace(
	last_cycle_start_time, # start of the last cycle
	simulation.total_time_len, # end of the last cycle
	1000, # number of recording timepoints
)
simulation.set_state_rec_timepoints(timepoints)

# snapshot_rec_timepoints
timepoints = [
	# start of anaerobic phase
	last_cycle_start_time + inflow_phase.time_len,
	# end of anaerobic phase / start of aerobic phase
	last_cycle_start_time + inflow_phase.time_len + anaerobic_phase.time_len,
	# end of aerobic phase
	last_cycle_start_time + inflow_phase.time_len + anaerobic_phase.time_len
		+ anaerobic_phase.time_len,
]
simulation.set_snapshot_rec_timepoints(timepoints)

## 5. Run simulation

After SBR, agent, and recording configurations are done, call `simulation.run()` to run the simulation. The simlation will internally validate itself and catch some ill-formed parameters.

Optionally, `simulation.get_run_duration()` can be used to show the wall-clock duration of the last run in miliseconds.

In [ ]:
simulation.run()

print("last run duration: %.3fsec" % (simulation.get_run_duration() / 1000))

## 6. Plot results

### 6.1 Plot environment state recording

In [ ]:
# retrieve recording data from the simulation by
env_state_rec = simulation.retrieve_env_state_rec()

# have a peek
env_state_rec

The recoding data is a 1-dim array with 4 fields: `volume`, `vfa_conc`, `op_conc`, and `is_aerobic`. Plotting the results can be easy, using `op_conc` as an example:

In [ ]:
matplotlib.pyplot.plot(simulation.get_state_rec_timepoints(), env_state_rec["op_conc"])
matplotlib.pyplot.ylim(0, None)
matplotlib.pyplot.xlabel("day")
matplotlib.pyplot.ylabel("op (mgP/L)");

### 6.2 Plot agent state recording

In [ ]:
# similarly, retrieve recording data by
agent_state_rec = simulation.retrieve_agent_state_rec()

# have a peek
agent_state_rec

It is 2-dim array with the first dimension being recording timepoints and the second being agent subtypes. Each recording has 5 fields: `biomass`, `rela_count`, `glycogen`, `pha`, and `polyp`, despite that some subtypes may not use all of these fields. The agent subtypes dimension corresponds to the subtypes' added order, represented by calling:

In [ ]:
subtypes = simulation.n_agent_by_subtype

subtypes

For example, to plot pao polyp during the last cycle:

In [ ]:
subtype_idx = 0 # plot the first added subtype

matplotlib.pyplot.plot(simulation.get_state_rec_timepoints(),
	agent_state_rec[:, subtype_idx]["polyp"])
matplotlib.pyplot.ylim(0, None)
matplotlib.pyplot.xlabel("day")
matplotlib.pyplot.ylabel("polyp (mgP/L)")
matplotlib.pyplot.title(subtypes[subtype_idx][0]);

### 6.3 Plot agent snapshot recording

In [ ]:
# similarly, retrieve recording data by
snapshot_rec = simulation.retrieve_snapshot_rec()

# have a peek
snapshot_rec

The snapshot recording data is a list of 2-dim arrays. Arrays represent the snapshots of subtypes in added order. The first dimension of each array represents the snapshot recording timepoints and the second dimension represents the agents in the subtype. For example, to find the snapshot of pao (first added subtype as index 0) at the end of anaerobic phase (the 3nd snapshot recording timepoints as index 2):

In [ ]:
snapshot_rec[0][2]

Below is an example to plot polyp distribution of intracellular polyp of pao at the end of aerobic phase:

In [ ]:
subtype_idx = 0 # the 1st added subtype (pao)
snapshot_idx = 2 # the 3rd snapshot recording (end of aerobic phase)
field = "polyp"

# use this utility function to esimate the distribution
pop_cumu, norm_content = iebpr.util.estimate_subtype_distrib_from_snapshot(
	snapshot_rec[subtype_idx][snapshot_idx], field=field,
)

matplotlib.pyplot.plot(pop_cumu, norm_content)
matplotlib.pyplot.xlim(0, 1)
matplotlib.pyplot.ylim(0, None)
matplotlib.pyplot.xlabel("population fraction")
matplotlib.pyplot.ylabel("%s content (mgP/mgCOD biomass)" % field)
matplotlib.pyplot.title("%s in %s at day %.2f" % (field, subtypes[subtype_idx][0],
	simulation.get_snapshot_rec_timepoints()[snapshot_idx])
);